# Installing, Cloning & Importing

In [ ]:
!pip -q install tensorflow==2.1

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 60.2MB/s 
     |████████████████████████████████| 450kB 74.0MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [ ]:
 !pip -q install keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=83f094097d77e6b748845888352095e7bcffccc519f388196ec80721fadbce72
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b53c4cbfed204210343122d686890a0c072e2418fc8d007665395f8ba15dfa2e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
# Update dataset: !git pull
!git clone https://github.com/casperbh96/COVID-19-Detection.git dataset

Cloning into 'dataset'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9745 (delta 4), reused 0 (delta 0), pack-reused 9736
Receiving objects: 100% (9745/9745), 151.88 MiB | 47.79 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2, time
import tensorflow as tf


tf.__version__

'2.1.0'

In [ ]:
import kerastuner

kerastuner.__version__

'1.0.1'

# Loading The Datasets

In [ ]:
covid_path = 'dataset/covid_dataset.csv'
covid_image_path = 'dataset/covid_adjusted/'

normal_path = 'dataset/normal_xray_dataset.csv'
normal_image_path = 'dataset/normal_dataset/'

covid_df = pd.read_csv(covid_path, usecols=['filename', 'finding'])
normal_df = pd.read_csv(normal_path, usecols=['filename', 'finding'])

normal_df = normal_df.head(99)

covid_df.head()

,filename,finding
0,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,COVID-19
1,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,COVID-19
2,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,COVID-19
3,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,COVID-19
4,nejmc2001573_f1a.jpeg.jpg,COVID-19


In [ ]:
covid_images = []
covid_labels = []

for index, row in covid_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = covid_image_path + filename

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    covid_images.append(image)
    covid_labels.append(label)

normal_images = []
normal_labels = []

for index, row in normal_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = normal_image_path + filename

    # temporary fix while we preprocess ALL the images
    if filename == '4c268764-b5e5-4417-85a3-da52916984d8.jpg':
        break

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    normal_images.append(image)
    normal_labels.append(label)

# normalize to interval of [0,1]
covid_images = np.array(covid_images) / 255

# normalize to interval of [0,1]
normal_images = np.array(normal_images) / 255

# Splitting Datasets Into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

# split into training(0.7) and testing(0.15) and validation(0.15)
covid_x_train, covid_x_val, covid_y_train, covid_y_val= train_test_split(
    covid_images, covid_labels, test_size=0.3)
covid_x_test, covid_x_val, covid_y_test, covid_y_val= train_test_split(
    covid_x_val, covid_y_val, test_size=0.5)

normal_x_train, normal_x_val, normal_y_train, normal_y_val = train_test_split(
    normal_images, normal_labels, test_size=0.3)
normal_x_test, normal_x_val, normal_y_test, normal_y_val = train_test_split(
    normal_x_val, normal_y_val, test_size=0.5)



X_train = np.concatenate((normal_x_train, covid_x_train), axis=0)
X_val = np.concatenate((normal_x_val, covid_x_val), axis=0)
X_test = np.concatenate((normal_x_test, covid_x_test), axis=0)
y_train = np.concatenate((normal_y_train, covid_y_train), axis=0)
y_val = np.concatenate((normal_y_val, covid_y_val), axis=0)
y_test = np.concatenate((normal_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_val = LabelBinarizer().fit_transform(y_val)
y_val = to_categorical(y_val)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)# split into training(0.7) and testing(0.15) and validation(0.15)
covid_x_train, covid_x_val, covid_y_train, covid_y_val= train_test_split(
    covid_images, covid_labels, test_size=0.3)
covid_x_test, covid_x_val, covid_y_test, covid_y_val= train_test_split(
    covid_x_val, covid_y_val, test_size=0.5)

normal_x_train, normal_x_val, normal_y_train, normal_y_val = train_test_split(
    normal_images, normal_labels, test_size=0.3)
normal_x_test, normal_x_val, normal_y_test, normal_y_val = train_test_split(
    normal_x_val, normal_y_val, test_size=0.5)



X_train = np.concatenate((normal_x_train, covid_x_train), axis=0)
X_val = np.concatenate((normal_x_val, covid_x_val), axis=0)
X_test = np.concatenate((normal_x_test, covid_x_test), axis=0)
y_train = np.concatenate((normal_y_train, covid_y_train), axis=0)
y_val = np.concatenate((normal_y_val, covid_y_val), axis=0)
y_test = np.concatenate((normal_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_val = LabelBinarizer().fit_transform(y_val)
y_val = to_categorical(y_val)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


# Modeling With Keras

In [ ]:
from tensorflow import keras

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten # Convert pools feature map into this large feature vector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D


In [ ]:
def build_model(hp):  # random search passes this hyperparameter() object 
    vggModel = VGG19(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
    outputs = vggModel.output
    
    m_filters = 256
    m_kernel_size = (1, 1)  # (3, 3) # (1, 1)
    m_pool_size = (2, 2)    # (2, 2) # (1, 1) # to reduce the computation time and potential overfitting
    # also pick out the most activated pixels and disgard the lower value pixels
    m_stride_size = (1, 1)  # (1, 1) # (2, 2)
    
    outputs = Conv2D(filters = m_filters, kernel_size = m_kernel_size, strides = m_stride_size, name = 'conv0')(outputs)
    outputs = BatchNormalization(axis = 3, name = 'bn0')(outputs)
    outputs = Activation('relu')(outputs) 
    outputs = MaxPooling2D(m_pool_size, strides = m_stride_size, name='max_pool0')(outputs) 

    outputs = Conv2D(filters = m_filters, kernel_size = m_kernel_size, strides = m_stride_size, name = 'conv1')(outputs)
    outputs = BatchNormalization(axis = 3, name = 'bn1')(outputs)
    outputs = Activation('relu')(outputs) 
    outputs = Dropout(0.5)(outputs)

    outputs = Conv2D(filters = m_filters, kernel_size = m_kernel_size, strides = m_stride_size, name = 'conv2')(outputs)
    outputs = BatchNormalization(axis = 3, name = 'bn2')(outputs)
    outputs = Activation('relu')(outputs) 
    outputs = MaxPooling2D(pool_size = m_pool_size, strides = m_stride_size, name='max_pool1')(outputs)

    outputs = Flatten(name="flatten")(outputs)
    outputs = Dropout(0.5)(outputs)
    outputs = Dense(2, activation="softmax")(outputs)

    model = Model(inputs = vggModel.input, outputs = outputs)

    for layer in vggModel.layers:
        layer.trainable = False

    model.compile(optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model
   
    

In [ ]:
import time
LOG_DIR = f"{int(time.time())}"

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,  # how many model variations to test?
    executions_per_trial=5,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)
# earlystop = EarlyStopping(monitor = 'loss',patience = 5, baseline = None)
tuner.search(train_aug.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    validation_steps=len(X_val) / 32,
                    steps_per_epoch=len(X_train) / 32,
                    epochs=100)

  ...
    to  
  ['...']
Train for 4.15625 steps, validate on 29 samples
Epoch 1/100
5/4 [====================================] - 2s 476ms/step - loss: 3.9057 - accuracy: 0.5564 - val_loss: 0.7942 - val_accuracy: 0.4828
Epoch 2/100
5/4 [====================================] - 2s 305ms/step - loss: 2.2021 - accuracy: 0.6015 - val_loss: 0.6309 - val_accuracy: 0.5862
Epoch 3/100
5/4 [====================================] - 2s 456ms/step - loss: 1.6488 - accuracy: 0.6391 - val_loss: 0.5534 - val_accuracy: 0.6897
Epoch 4/100
5/4 [====================================] - 1s 273ms/step - loss: 1.0685 - accuracy: 0.6165 - val_loss: 0.9137 - val_accuracy: 0.4828
Epoch 5/100
5/4 [====================================] - 1s 269ms/step - loss: 0.8421 - accuracy: 0.6842 - val_loss: 1.1789 - val_accuracy: 0.4828
Epoch 6/100
5/4 [====================================] - 1s 288ms/step - loss: 0.6231 - accuracy: 0.7669 - val_loss: 1.0710 - val_accuracy: 0.4828
Epoch 7/100
5/4 [============================

  ...
    to  
  ['...']
Train for 4.15625 steps, validate on 29 samples
Epoch 1/100
5/4 [====================================] - 2s 478ms/step - loss: 1.0731 - accuracy: 0.5338 - val_loss: 0.7169 - val_accuracy: 0.5172
Epoch 2/100
5/4 [====================================] - 1s 299ms/step - loss: 1.7413 - accuracy: 0.5338 - val_loss: 0.6854 - val_accuracy: 0.5862
Epoch 3/100
5/4 [====================================] - 1s 275ms/step - loss: 1.1224 - accuracy: 0.5940 - val_loss: 0.6973 - val_accuracy: 0.5172
Epoch 4/100
5/4 [====================================] - 1s 287ms/step - loss: 1.2395 - accuracy: 0.5714 - val_loss: 0.6776 - val_accuracy: 0.5517
Epoch 5/100
5/4 [====================================] - 1s 273ms/step - loss: 1.1622 - accuracy: 0.5414 - val_loss: 0.6745 - val_accuracy: 0.5517
Epoch 6/100
5/4 [====================================] - 1s 271ms/step - loss: 0.8469 - accuracy: 0.6391 - val_loss: 0.6766 - val_accuracy: 0.5517
Epoch 7/100
5/4 [============================

  ...
    to  
  ['...']
Train for 4.15625 steps, validate on 29 samples
Epoch 1/100
5/4 [====================================] - 2s 489ms/step - loss: 2.4972 - accuracy: 0.4511 - val_loss: 0.7242 - val_accuracy: 0.4828
Epoch 2/100
5/4 [====================================] - 2s 305ms/step - loss: 1.7492 - accuracy: 0.5414 - val_loss: 0.7128 - val_accuracy: 0.5172
Epoch 3/100
5/4 [====================================] - 2s 457ms/step - loss: 1.8396 - accuracy: 0.4812 - val_loss: 0.6780 - val_accuracy: 0.5517
Epoch 4/100
5/4 [====================================] - 1s 286ms/step - loss: 1.1484 - accuracy: 0.6090 - val_loss: 0.6988 - val_accuracy: 0.5172
Epoch 5/100
5/4 [====================================] - 2s 309ms/step - loss: 0.9793 - accuracy: 0.6767 - val_loss: 0.6561 - val_accuracy: 0.6207
Epoch 6/100
5/4 [====================================] - 1s 285ms/step - loss: 1.3757 - accuracy: 0.6316 - val_loss: 0.6556 - val_accuracy: 0.5517
Epoch 7/100
5/4 [============================

INFO:tensorflow:Oracle triggered exit


In [ ]:
for layer in tuner.get_best_models()[0].layers:
  if 'MaxPool' in str(layer):
    print('L:{}  S:{}'.format(str(layer), str(layer.output.shape)))
    print(layer.get_config())

L:<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5c1d51b70>  S:(None, 112, 112, 64)
{'name': 'block1_pool', 'trainable': False, 'dtype': 'float32', 'pool_size': (2, 2), 'padding': 'valid', 'strides': (2, 2), 'data_format': 'channels_last'}
L:<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5c80485f8>  S:(None, 56, 56, 128)
{'name': 'block2_pool', 'trainable': False, 'dtype': 'float32', 'pool_size': (2, 2), 'padding': 'valid', 'strides': (2, 2), 'data_format': 'channels_last'}
L:<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5be77e470>  S:(None, 28, 28, 256)
{'name': 'block3_pool', 'trainable': False, 'dtype': 'float32', 'pool_size': (2, 2), 'padding': 'valid', 'strides': (2, 2), 'data_format': 'channels_last'}
L:<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5c88f4cc0>  S:(None, 14, 14, 512)
{'name': 'block4_pool', 'trainable': False, 'dtype': 'float32', 'pool_size': (2, 2), 'padding': 'valid', 'strides':

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'learning_rate': 0.0011060200588733464}

In [ ]:
tuner.get_best_models(num_models=1)[0]
# <tensorflow.python.keras.engine.training.Model at 0x7fe5c0fc0b00>

In [ ]:
tuner.get_best_models()[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
modelBest = tuner.get_best_models(num_models=1)[0]

In [ ]:
for layer in modelBest.layers:
  print('L:{}  S:{}'.format(str(layer), str(layer.output.shape)))
  print(layer.output)

L:<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fe5c0389eb8>  S:(None, 224, 224, 3)
Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32)
L:<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5c1124ba8>  S:(None, 224, 224, 64)
Tensor("block1_conv1/Identity:0", shape=(None, 224, 224, 64), dtype=float32)
L:<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5c031b550>  S:(None, 224, 224, 64)
Tensor("block1_conv2/Identity:0", shape=(None, 224, 224, 64), dtype=float32)
L:<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe5c0425710>  S:(None, 112, 112, 64)
Tensor("block1_pool/Identity:0", shape=(None, 112, 112, 64), dtype=float32)
L:<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5c03f45c0>  S:(None, 112, 112, 128)
Tensor("block2_conv1/Identity:0", shape=(None, 112, 112, 128), dtype=float32)
L:<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe5c03eea20>  S:(None, 112, 112, 12

In [ ]:

history = modelBest.fit(train_aug.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    validation_steps=len(X_val) / 32,
                    steps_per_epoch=len(X_val) / 32,
                    epochs=200)



  ...
    to  
  ['...']
Train for 0.90625 steps, validate on 29 samples
Epoch 1/200
1/0 [=================================] - 1s 1s/step - loss: 0.5678 - accuracy: 0.8750 - val_loss: 0.7858 - val_accuracy: 0.8621
Epoch 2/200
1/0 [=================================] - 1s 701ms/step - loss: 0.6530 - accuracy: 0.8750 - val_loss: 0.6547 - val_accuracy: 0.8276
Epoch 3/200
1/0 [=================================] - 1s 597ms/step - loss: 0.3339 - accuracy: 0.9062 - val_loss: 0.5976 - val_accuracy: 0.8276
Epoch 4/200
1/0 [=================================] - 1s 607ms/step - loss: 0.1895 - accuracy: 0.9375 - val_loss: 0.5905 - val_accuracy: 0.8276
Epoch 5/200
1/0 [=================================] - 0s 322ms/step - loss: 2.4711 - accuracy: 0.6000 - val_loss: 0.6754 - val_accuracy: 0.7931
Epoch 6/200
1/0 [=================================] - 0s 354ms/step - loss: 1.2145e-04 - accuracy: 1.0000 - val_loss: 0.8200 - val_accuracy: 0.8276
Epoch 7/200
1/0 [=================================] - 1s 632ms

In [ ]:
acc_scores = list()
acc = modelBest.evaluate(X_test, y_test, verbose=0)
acc_scores.append(acc[1])
print('Accuracy scores: ', acc_scores)
print(acc)

Accuracy scores:  [0.7586207]
[1.4521515369415283, 0.7586207]


In [ ]:
y_pred = modelBest.predict(X_test, batch_size=32)
y_pred_covid = modelBest.predict(covid_x_test, batch_size=32)

In [ ]:
y_test_covid = LabelBinarizer().fit_transform(covid_y_test)
y_test_covid = to_categorical(y_test_covid)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('dark_background')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('dark_background')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('dataset/normal_dataset/0b421aea-6e0f-4faf-a4bf-4a82445d0e35.jpg', target_size = (224,224))
#test_image = image.load_img('dataset/covid_adjusted/covid-19-pneumonia-rapidly-progressive-12-hours.jpg.jpg', target_size = (224,224))


test_image = image.img_to_array(test_image)
test_image = test_image *1./255

test_image = np.expand_dims(test_image, axis = 0)
test_image = np.vstack([test_image ])
result = modelBest.predict(test_image,batch_size=32)
print(result)

In [ ]:
import matplotlib.image as mpimg

img=mpimg.imread('dataset/normal_dataset/0b421aea-6e0f-4faf-a4bf-4a82445d0e35.jpg')
#img=mpimg.imread('dataset/covid_adjusted/covid-19-pneumonia-rapidly-progressive-12-hours.jpg.jpg')
imgplot = plt.imshow(img)
plt=plt.title('Chest X-ray ')

In [ ]:
if result[0][0]<result[0][1]:

    prediction = 'normal'
else:
    prediction = 'covid'
    
print("AI's prediction is: "+ prediction)